# Определение токсичных комментариев

Дан крупный набор комментариев на английском с разметкой о токсичности.
Необходимо построить модель классификации со значением метрики качества *F1* не меньше 0.75. 

Структура исходных данных:
- text - текст комментария
- toxic — целевой признак (1 - токсичный комментарий, 0 - обычный комментарий)

## 1. Подготовка

In [1]:
#Импорт необходимых библиотек:
import pandas as pd
import numpy as np

import string
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack

from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Считывание файла с данными, первые несколько строк таблицы:
comments_data = pd.read_csv('/datasets/toxic_comments.csv')
comments_data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
#Основная информация:
comments_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
#Пример комментария:
comments_data.loc[1000]['text']

"Rex Mundi \n\nI've created a stub on Rex Mundi at Rex Mundi High School.  Only thing I know about it is that both my Aunt Donna and Bob Griese went there.  Please add anything you might know about it.\n\nBTW, my dad was a Panther; I live in Princeton myself."

Данные содержат 159571 комментарий с пометкой о токсичности. Пропусков в данных нет. В комментариях есть прописные и строчные буквы, алгоритмы токенизации могут принять слова с буквами в разных регистрах за разные слова, поэтому все буквы я приведу к нижнему регистру. Также можно заметить, что в сообщениях часто встречается последовательность символов \n - скорее всего это специальные символы, отвечающие за перенос строки, их лучше удалить.

In [5]:
#Замена символов \n на пробел (пробел, а не пустота, чтобы слова не слипались):
comments_data['text'] = comments_data['text'].apply(lambda x: x.replace('\n', ' '))

Прежде чем заменять все большие буквы на маленькие, посмотрю нет ли зависимости между большим количеством прописных букв в комментарии и токсичностью - оскорбительные комментарии могут писаться капслоком. Кроме того, хочется проверить как длина сообщения соотносится с токсичностью - возможно грубые комментарии очень короткие.

<font color='orange'> 
*Когда я выполнял этот проект, мне показалось хорошей идеей использовать коэффициент корреляции Пирсона. Теперь я понимаю, что использовать его для категориальных данных некорректно*
</font> 

In [6]:
#Добавление столбца с длиной сообщения:
comments_data['len'] = comments_data['text'].apply(len)

#Коэффициент корреляции между длиной сообщения и его токсичностью:
comments_data['len'].corr(comments_data['toxic'])

-0.0516960635833761

Длина сообщения почти не коррелирует с токсичностью.

In [7]:
#Вывод некоторых коротких оскорбительных комментариев. Можно найти какие-нибудь неочевидные сходства таких комментариев.
comments_data[(comments_data['len'] < 20) & (comments_data['toxic'] == 1)].head(15)

,text,toxic,len
2981,cocksucking bastard,1,19
17973,ya mum fucks ya,1,17
19789,One word: freaks. -,1,19
22778,Fuck you Fuck you,1,19
23611,questions cuntface!,1,19
26794,YOU ARE AN ASSHOLE,1,18
33771,alex fuck you ...,1,19
46949,You're such a slut.,1,19
60495,WHY YOU SO PIG??,1,16
60971,Get lost Get lost,1,19


In [8]:
#Для сравнения вывод корректных коротких комментариев:
comments_data[(comments_data['len'] < 20) & (comments_data['toxic'] == 0)].head(15)

,text,toxic,len
899,I've just seen that,0,19
1275,Is it still a stub?,0,19
2461,- unsigned comment,0,18
2506,I understand now.,0,17
2823,"Sure, goodnight! )",0,18
3354,Agreed and done.,0,16
3595,comments on draft 2,0,19
3638,September 2008,0,14
4424,And quote from it,0,17
5145,3rd Unblock Request,0,19


Действительно, на этой небольшой выборке комментариев видно, что в токсичных комментариях частенько используется капслок. В корректных комментариях в конце пользователи могут ставить ')' или смайлик с этой скобкой. Можно предположить, что в токсичных комментариях чаще будет использоваться восклицательный знак. Добавлю столбцы с новыми признаками и проверю корреляцию их с токсичностью.

In [9]:
%%time
#Функция расчета соотношения количества прописных букв и всех символов в тексте:
def upper_case_ratio(entry):
    
    length = entry['len']
    upper = 0
    text_list = list(entry['text'])
    
    for i in text_list:
        if i.isupper():
            upper += 1
            
    return upper/length

#Добавление соотношения прописных букв и символов в тексте к таблице:
comments_data['uppercase_ratio'] = comments_data.apply(upper_case_ratio, axis=1)

#Коэффициент корреляции между новым признаком и токсичностью:
comments_data['uppercase_ratio'].corr(comments_data['toxic'])

CPU times: user 11.9 s, sys: 19.7 ms, total: 11.9 s
Wall time: 11.9 s


0.2153173791222518

Есть положительная корреляция между новым признаком и токсичностью комментария.

In [10]:
#Вывод комментариев с самым большим соотношением больших букв к символам:
comments_data.sort_values(by='uppercase_ratio', ascending=False).head(15)

,text,toxic,len,uppercase_ratio
11796,==U R GAY== FUCKFUCKFUCKFUCKFUCKFUCKFUCKFUCK...,1,4969,0.998189
34637,shut up you cunt WWWWWWWWWWWWWWWWWWWWWWWWWWWWW...,1,4177,0.995930
29580,YO MOMA IS COOL AND FARTS 247 HI BYE GIGOWNG...,1,459,0.969499
106067,FFFFFFFFFFFFFFFUUUUUUUUUUUUUUUUUUUUUUUCCCCCCCC...,1,86,0.965116
71584,MRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR...,0,189,0.962963
21366,I SAID GOOD DAY BIATCH DONT BREAK WP:3RR BIATC...,1,342,0.959064
40851,Discussion destroying CRAP topic It seems th...,1,4957,0.951584
22657,""" find the """"W"""" MMMMMMMMMMMMMMMMMMMMMMMMM...",0,422,0.950237
113646,HA GAAAAAAAAAYYYYY,1,18,0.944444
108142,BLAHBLAHBLAHBLAHNLAHNLAHBLAHBLHA AND STOP CHAN...,1,102,0.931373


In [11]:
#И для сравнения вывод комментариев без больших букв:
comments_data.sort_values(by='uppercase_ratio').head(15)

,text,toxic,len,uppercase_ratio
79785,(who atleast had a degree in history),0,37,0.0
5947,what's with undoing those changes?,0,34,0.0
23164,dick shit cock fuck you are a fag you are a...,1,207,0.0
23181,they're back! anyone have a screencap from ar...,0,59,0.0
93601,its not me whos doing this. believe me.,0,39,0.0
93600,because he is a punk hu loves wow he is a big ...,1,58,0.0
93598,su.cked my c.ock last night,1,27,0.0
93588,co/ck - pen-i.s. international,1,30,0.0
142615,inject energy into these arrangements making t...,0,56,0.0
93586,{{unblock|yo,0,12,0.0


Видно, что новый признак неплохо делит комментарии на токсичные и корректные. У меня появилось еще несколько предположений:
- В токсичных комментариях много повторов одного символа подряд
- В токсичных комментариях оскорбительные слова маскируются - буквы в нем перемежаются знаками препинания и особыми символами - их нужно удалить, чтобы токенизация сработала лучше
- Можно "схлопнуть" комментарии за счет повторяющихся символов - так, чтобы символы не повторялись более двух раз подряд. В дальнейшем это также положительно скажется на создании токенов

In [12]:
%%time
#Функция для расчета соотношения повторяющихся символов и всех символов в тексте:
def repeat_letters_ratio(entry):
    
    length = entry['len']
    repeated = 0
    text_list = list(entry['text'])
    
    for i in range(len(text_list)-1):
        if text_list[i] == text_list[i+1]:
            repeated += 1
            
    return repeated/length

#Применение функции и создание нового признака:
comments_data['repeat_letters_ratio'] = comments_data.apply(repeat_letters_ratio, axis=1)

CPU times: user 15 s, sys: 59.4 ms, total: 15.1 s
Wall time: 15.1 s


In [13]:
%%time
#Функция для расчета соотношения восклицательных знаков и всех символов в тексте:
def exclamation_point_ratio(entry):
    
    length = entry['len']
    exclamation_point = 0
    text_list = list(entry['text'])
    
    for i in text_list:
        if i == '!':
            exclamation_point += 1
            
    return exclamation_point/length

#Применение функции:
comments_data['exclamation_point_ratio'] = comments_data.apply(exclamation_point_ratio, axis=1)

CPU times: user 9.39 s, sys: 43.8 ms, total: 9.43 s
Wall time: 9.45 s


In [14]:
%%time
#Функция для расчета соотношения закрывающей скобки и всех символов в тексте:
def closing_bracket_ratio(entry):
    
    length = entry['len']
    closing_bracket = 0
    text_list = list(entry['text'])
    
    for i in text_list:
        if i == ')':
            closing_bracket += 1
            
    return closing_bracket/length

#Применение функции:
comments_data['closing_bracket_ratio'] = comments_data.apply(closing_bracket_ratio, axis=1)

CPU times: user 9.58 s, sys: 35.9 ms, total: 9.61 s
Wall time: 9.63 s


In [15]:
#Коэффициенты корреляции новых признаков и токсичности:
comments_data.corr()['toxic']

toxic                      1.000000
len                       -0.051696
uppercase_ratio            0.215317
repeat_letters_ratio       0.110791
exclamation_point_ratio    0.129425
closing_bracket_ratio     -0.075592
Name: toxic, dtype: float64

Задамся порогом коэффициента корреляции 0.1 и удалю признаки, показавшие небольшую связь с целевым признаком. И продолжу обработку данных.

In [16]:
#Удаление лишних признаков:
comments_data.drop(['len', 'closing_bracket_ratio'], axis=1, inplace=True)

#Удаление знаков пунктуации из комментариев (и склеивание зашифрованных слов):
tt = str.maketrans(dict.fromkeys(string.punctuation))
comments_data['text'] = comments_data['text'].apply(lambda x: x.translate(tt))

#Перевод букв в нижний регистр:
comments_data['text'] = comments_data['text'].apply(lambda x: x.lower())

In [17]:
%%time
#Функция для удаления символов, повторяющихся подряд более 2 раз:
def max_repeat_letters_2(string):
    
    if len(string) < 3:
        return string
    
    new_str = ''
    
    for i in range(len(string)-2):
        
        i1 = i
        i2 = i + 1
        i3 = i + 2
        
        if string[i1] != string[i2]:
            new_str = new_str + string[i1]
        else:
            if string[i2] != string[i3]:
                new_str = new_str + string[i1]
                
    new_str = new_str + string[-2] + string[-1]
                
    return new_str

#Применение функции к комментариям:
comments_data['text'] = comments_data['text'].apply(max_repeat_letters_2)

CPU times: user 19.9 s, sys: 31.9 ms, total: 19.9 s
Wall time: 19.9 s


In [18]:
%%time
#Функция для удаления всех символов, кроме латинских букв в тексте:
def only_latin(string):
    
    words = re.sub(r'[^a-zA-Z ]', '', string)
    
    return words

#Применение функции:
comments_data['text'] = comments_data['text'].apply(only_latin)

CPU times: user 819 ms, sys: 4.03 ms, total: 823 ms
Wall time: 833 ms


In [19]:
%%time
#Стемминг каждого слова в тексте:
def stemming(text):
    
    englishStemmer = SnowballStemmer('english')
    
    splitted = text.split()
    
    stemmed_splitted = []
    
    for word in splitted:
        
        stemmed_splitted.append(englishStemmer.stem(word))
        
    stemmed_text = ' '.join(stemmed_splitted)
        
    return stemmed_text

#Применение функции:
comments_data['stemmed_text'] = comments_data['text'].apply(lambda x: stemming(x))

CPU times: user 2min 32s, sys: 216 ms, total: 2min 32s
Wall time: 2min 32s


In [20]:
#Несколько записей обработанных данных:
comments_data.head()

,text,toxic,uppercase_ratio,repeat_letters_ratio,exclamation_point_ratio,stemmed_text
0,explanation why the edits made under my userna...,0,0.064394,0.007576,0.000000,explan whi the edit made under my usernam hard...
1,daww he matches this background colour im seem...,0,0.071429,0.035714,0.008929,daww he match this background colour im seem s...
2,hey man im really not trying to edit war its j...,0,0.017167,0.012876,0.000000,hey man im realli not tri to edit war it just ...
3,more i cant make any real suggestions on impr...,0,0.017685,0.020900,0.000000,more i cant make ani real suggest on improv i ...
4,you sir are my hero any chance you remember wh...,0,0.029851,0.000000,0.000000,you sir are my hero ani chanc you rememb what ...


In [21]:
#Доля токсичных комментариев в выборке:
len(comments_data[comments_data['toxic'] == 1])/len(comments_data)

0.10167887648758234

Исходную выборку теперь нужно разделить на обучающую и тестовую. Преобразования, производимые до этого, не зависели от того скрыта ли тестовая выборка. Далее же признаки будут создаваться на основе известной (обучающей) выборки, а к тестовой будут только применяться подготовленные алгоритмы.

In [22]:
#Отделение признаков от целевого признака:
features = comments_data.drop('toxic', axis=1)
target = comments_data['toxic']

In [23]:
#Разделение исходной выборки на обучающую и тестовую с одинаковой долей токсичных комментариев в обеих выборках:
features_train_big, features_test, target_train_big, target_test = (
    train_test_split(features, target, test_size=0.1, stratify=target, random_state=100)
)

#Гиперпараметры моделей буду подбирать с помощью валидационной выборки:
features_train_small, features_valid, target_train_small, target_valid = (
    train_test_split(features_train_big, target_train_big, test_size=0.15, stratify=target_train_big, random_state=200)
)

Я изучил данные в выборке, подготовил комментарии к дальнейшей обработке и выделил три дополнительных признака для обучения. Бегло рассмотрев комментарии, как токсичные, так и корректные, я могу предположить, что проще всего будет находить токсичные: в них часто встречаются одни и те же слова и словосочетания. Корректные же комментарии гораздо более разнообразны и их в выборке большинство - около 90%. Мне кажется, что n-граммы для решения задачи можно не использовать, будет достаточно tf-idf для некоторых слов. Но если n-граммы и добавить для обучения, то можно ограничиться двумя-тремя словами в них.

Лемматизацию средствами nltk провести не удалось, потому что модуль wordnetlemmatizer работает только если корректно проставить теги каждому слову. Ограничусь стеммингом с помощью SnowballStemmer из той же библиотеки.

In [24]:
#Загрузка "стоп-слов", создание счетчика tf-idf:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
%%time
#Подготовка признаков tf-idf для обучения, тюнинга и валидации моделей:
tf_idf_train_small = count_tf_idf.fit_transform(features_train_small['stemmed_text'])
tf_idf_valid = count_tf_idf.transform(features_valid['stemmed_text'])

#Подготовка признаков tf-idf для финального тестирования
tf_idf_train_big = count_tf_idf.fit_transform(features_train_big['stemmed_text'])
tf_idf_test = count_tf_idf.transform(features_test['stemmed_text'])

CPU times: user 21 s, sys: 180 ms, total: 21.1 s
Wall time: 21.3 s


In [26]:
#Добавление в выборки признаков, созданных в процессе обработки данных:

features_train_small = (
    hstack([tf_idf_train_small, csr_matrix(np.array(features_train_small[['uppercase_ratio',
                                                                          'repeat_letters_ratio',
                                                                          'exclamation_point_ratio']]))], format='csr')
)

features_valid = (
    hstack([tf_idf_valid, csr_matrix(np.array(features_valid[['uppercase_ratio',
                                                              'repeat_letters_ratio',
                                                              'exclamation_point_ratio']]))], format='csr')
)

features_train_big = (
    hstack([tf_idf_train_big, csr_matrix(np.array(features_train_big[['uppercase_ratio',
                                                                      'repeat_letters_ratio',
                                                                      'exclamation_point_ratio']]))], format='csr')
)

features_test = (
    hstack([tf_idf_test, csr_matrix(np.array(features_test[['uppercase_ratio',
                                                            'repeat_letters_ratio',
                                                            'exclamation_point_ratio']]))], format='csr')
)

Можно приступать к обучению. Пробные запуски показали, что модели с "деревьями" обучаются очень долго, а их точность далека до точности линейной регрессии, поэтому далее я поработаю только с линейными моделями.

## 2. Обучение

In [27]:
%%time
#Обучение и валидация модели логистической регрессии:
model_lr = LogisticRegression()
model_lr.fit(features_train_small, target_train_small)
predictions = model_lr.predict(features_valid)
f1_score(target_valid, predictions)

CPU times: user 6.19 s, sys: 4.37 s, total: 10.6 s
Wall time: 10.6 s


0.7542671362774316

In [28]:
%%time
#Обучение и валидация модели-перцептрона:
model_perc = Perceptron()
model_perc.fit(features_train_small, target_train_small)
predictions = model_perc.predict(features_valid)
f1_score(target_valid, predictions)

CPU times: user 464 ms, sys: 96 ms, total: 560 ms
Wall time: 577 ms


0.7397504456327985

In [29]:
%%time
#Обучение и валидация модели RidgeClassifier:
model_rc = RidgeClassifier()
model_rc.fit(features_train_small, target_train_small)
predictions = model_rc.predict(features_valid)
f1_score(target_valid, predictions)

CPU times: user 2.76 s, sys: 7.94 ms, total: 2.76 s
Wall time: 2.78 s


0.7104966139954852

Лучшая метрика f1 была достигнута с помощью логистической регрессии, к тому же метрика на неполной выборке выше 0.75, чего нужно добиться на финальном тестировании. Две другие модели показали несколько худший результат. Возможно удастся улучшить метрику, если объединить три эти модели с помощью VotingClassifier - если большинство моделей предсказывают класс 1, то это будет ответом VotingClassifier'а.

In [30]:
%%time
#Обучение и валидация модели VotingClassifier:
model_vc = VotingClassifier(estimators=[('lr', model_lr), ('perc', model_perc), ('rc', model_rc)],
                            voting='hard')

model_vc.fit(features_train_small, target_train_small)
predictions = model_vc.predict(features_valid)
f1_score(target_valid, predictions)

CPU times: user 9.94 s, sys: 4.38 s, total: 14.3 s
Wall time: 14.3 s


0.7656040717921243

Действительно, метрика немного подросла при использовании сразу трех моделей. Некоторые ошибки логистической регрессии исправились совместным "голосованием" двух других моделей.

Проведу финальную проверку, обучив модель на полной обучающей выборке и проверив на тестовой.

In [31]:
%%time
#Финальное тестирование модели:
model_vc.fit(features_train_big, target_train_big)
predictions = model_vc.predict(features_test)
f1_score(target_test, predictions)

CPU times: user 11.5 s, sys: 3.97 s, total: 15.4 s
Wall time: 15.4 s


0.7734375000000001

F-мера выше порогового значения - модель пригодна для поиска токсичных комментариев.

## 3. Выводы

В ходе выполнения задания мне нужно было обработать исходные текстовые данные и подготовить их к обучению. Я постарался избавиться от многих проблем, способных снизить качество моделей: удалил все лишние символы, привел буквы к нижнему регистру, удалил последовательности одних и тех же букв и пр. В комментариях после обработки всё равно остаются некоторые проблемы: слипшиеся слова, сдвоенные буквы, орфографические ошибки - из-за них вычисленные tf-idf не равны истинным значениям. Скорее всего я не заметил еще какие-то особенности текстов, но решение всех проблем заняло бы очень много времени. Вместе с обработкой я выделил три дополнительных признака, не связанных с частотой слов в комментариях, надеюсь они хотя бы немного улучшили итоговую метрику.

Сделав несколько пробных прогонов я заметил, что модели с "деревьями" обучаются гораздо дольше линейных моделей, а точность их предсказаний ниже. Поэтому я решил обучать только линейные модели. В конце концов обученные модели я объединил в классификатор, предсказывающий класс по "голосованию" трех линейных моделей. В ТЗ установлена метрика F1 и пороговое значение 0.75. Финальный классификатор достигает F1 примерно равной 0,773 (при тестовой выборке размером 1/10 от исходной). Теперь модель можно испытать на новых комментариях.